In [13]:
!pip install sunpy[all]

from IPython.display import clear_output
clear_output()

In [14]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

from sunpy.net import Fido, attrs as a
from sunpy.time import TimeRange

from datetime import datetime
from datetime import timedelta
import requests

import warnings
warnings.filterwarnings('ignore')

from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
# Fetching the data from DONKI
donki_flare = pd.read_json("https://kauai.ccmc.gsfc.nasa.gov/DONKI/WS/get/FLR?startDate=2010-05-01&endDate=2024-12-31")
donki_cme = pd.read_json("https://kauai.ccmc.gsfc.nasa.gov/DONKI/WS/get/CME?startDate=2010-05-01&endDate=2024-12-31")

In [ ]:
print("Donki Flare data: \nShape",donki_flare.shape)
print("Columns:",donki_flare.columns)

print("\nDonki CME data: \nShape",donki_cme.shape)
print("Columns:",donki_cme.columns)

In [ ]:
# Extracting the important columns
donki_flare = donki_flare[['beginTime','peakTime','endTime','classType','activeRegionNum','linkedEvents']]
donki_cme = donki_cme[['startTime','sourceLocation','activeRegionNum','linkedEvents','cmeAnalyses']]

In [ ]:
# Only Flares data
flares = donki_flare[donki_flare["linkedEvents"].isna()]
flares.reset_index(drop=True,inplace=True)
flares

In [ ]:
# Flare Associated with CME & SEP data
flare_linked = donki_flare[donki_flare["linkedEvents"].notna()]
flare_linked.reset_index(drop=True,inplace=True)
flare_linked

In [ ]:
# Only CME
cme = donki_cme[donki_cme["linkedEvents"].isna() & donki_cme["activeRegionNum"].notna()]
cme.reset_index(drop=True,inplace=True)
cme

In [15]:
# Changing the peaktime to datetime format
from datetime import datetime

def convert_to_datetime(tstr):
    year = int(tstr[:4])
    month = int(tstr[5:7])
    day = int(tstr[8:10])
    hour = int(tstr[11:13])
    minute = int(tstr[14:16])
    return datetime(year, month, day, hour, minute)

In [ ]:
for i in range(flares.shape[0]):
    flares['peakTime'].iloc[i] = convert_to_datetime(flares['peakTime'].iloc[i])
for i in range(cme.shape[0]):
    cme['startTime'].iloc[i] = convert_to_datetime(cme['startTime'].iloc[i])
for i in range(flare_linked.shape[0]):
    flare_linked['peakTime'].iloc[i] = convert_to_datetime(flare_linked['peakTime'].iloc[i])

### Validation using GOES data

#### Flare Only

In [12]:
# Case 1: filling nan ARs
number_of_donki_mistakes = 0  # count the number of DONKI mistakes
# create an empty array to hold row numbers to drop at the end
flares_drops = []

for i in range(flares.shape[0]):
    if (np.isnan(flares.loc[i]['activeRegionNum'])):
        time = flares['peakTime'].iloc[i]
        time_range = TimeRange(time, time)
        listofresult = Fido.search(a.Time(time_range),a.hek.EventType("FL"),a.hek.OBS.Observatory == "GOES")

        if len(listofresult["hek"]) == 0:
            print(flares.loc[i]['classType'], "has no match in the GOES flare database ; dropping row.")
            flares_drops.append(i)
            number_of_donki_mistakes += 1
            continue
        else:
            if (listofresult["hek"]['ar_noaanum'] == 0).all():
                print(flares.loc[i]['activeRegionNum'], flares.loc[i]
                    ['classType'], "has no match in the GOES flare database ; dropping row.")
                flares_drops.append(i)
                number_of_donki_mistakes += 1
                continue
            else:
                print("Missing NOAA number:", flares['activeRegionNum'].iloc[i], flares['classType'].iloc[i],
                    flares['peakTime'].iloc[i], "should be", listofresult[0]['ar_noaanum'][0], "; changing now.")
                flares['activeRegionNum'].iloc[i] = listofresult[0]['ar_noaanum']
                number_of_donki_mistakes += 1

# Drop the rows for which there is no active region number in both the DONKI and GOES flare databases
flares = flares.drop(flares_drops)
flares = flares.reset_index(drop=True)
print('There are', number_of_donki_mistakes, 'DONKI mistakes so far.')

Missing NOAA number: nan X1.5 2011-03-09 23:23:00 should be 11166 ; changing now.
Missing NOAA number: nan M8.8 2011-09-25 04:50:00 should be 11302 ; changing now.
Missing NOAA number: nan M5.7 2012-05-10 04:18:00 should be 11476 ; changing now.
Missing NOAA number: nan M5.6 2012-07-02 10:52:00 should be 11515 ; changing now.
Missing NOAA number: nan M5.3 2012-07-04 09:55:00 should be 11515 ; changing now.
Missing NOAA number: nan M6.1 2012-07-05 11:44:00 should be 11515 ; changing now.
Missing NOAA number: nan M5.5 2012-08-18 01:02:00 should be 11548 ; changing now.
Missing NOAA number: nan M9.1 2012-10-20 18:14:00 should be 11598 ; changing now.
Missing NOAA number: nan M6.0 2012-11-13 02:04:00 should be 11613 ; changing now.
nan M1.7 has no match in the GOES flare database ; dropping row.
Missing NOAA number: nan M5.7 2013-05-03 17:32:00 should be 11739 ; changing now.
nan M1.1 has no match in the GOES flare database ; dropping row.
Missing NOAA number: nan C3.6 2014-04-22 05:47:00 

In [13]:
# Grab all the data from the GOES database
t_start = "2010-05-01"
t_end = "2024-12-31"
time_range = TimeRange(t_start, t_end)
listofresults = Fido.search(a.Time(time_range),a.hek.EventType("FL"),a.hek.OBS.Observatory == "GOES")
print('Grabbed all the GOES data; there are', len(listofresults["hek"]), 'events.')

Grabbed all the GOES data; there are 27201 events.


In [14]:
# Case 2: Updating Incorrect AR
peak_times_noaa = [item["event_peaktime"] for item in listofresults["hek"]]

for i in range(flares.shape[0]):
    # check if a particular DONKI flare peak time is also in the NOAA database
    peak_time_donki = flares['peakTime'].iloc[i]
    if peak_time_donki in peak_times_noaa:
        index = peak_times_noaa.index(peak_time_donki)
    else:
        continue
    # ignore NOAA active region numbers equal to zero
    if (listofresults["hek"][index]['ar_noaanum'] == 0):
        continue
    # if yes, check if the DONKI and NOAA active region numbers match up for this peak time
    # if they don't, flag this peak time and replace the DONKI number with the NOAA number
    if (listofresults["hek"][index]['ar_noaanum'] != int(flares['activeRegionNum'].iloc[i])):
        print('Messed up NOAA number:', int(flares['activeRegionNum'].iloc[i]), flares['classType'].iloc[i],
              flares['peakTime'].iloc[i], "should be", listofresults["hek"][index]['ar_noaanum'], "; changing now.")
        flares['activeRegionNum'].iloc[i] = listofresults["hek"][index]['ar_noaanum']
        number_of_donki_mistakes += 1
print('There are', number_of_donki_mistakes, 'DONKI mistakes so far.')

Messed up NOAA number: 11968 M1.3 2014-02-02 14:06:00 should be 11967 ; changing now.
Messed up NOAA number: 11967 M3.8 2014-02-04 01:23:00 should be 11968 ; changing now.
Messed up NOAA number: 12237 M1.3 2014-12-19 09:44:00 should be 12242 ; changing now.
Messed up NOAA number: 12322 M1.2 2015-04-21 22:01:00 should be 12325 ; changing now.
Messed up NOAA number: 12304 M2.2 2015-08-22 13:23:00 should be 12403 ; changing now.
Messed up NOAA number: 12304 M3.5 2015-08-22 21:24:00 should be 12403 ; changing now.
Messed up NOAA number: 12415 M1.5 2015-09-20 05:03:00 should be 12420 ; changing now.
Messed up NOAA number: 12423 M1.9 2015-09-27 10:40:00 should be 12422 ; changing now.
Messed up NOAA number: 12423 M1.1 2015-09-29 03:43:00 should be 12422 ; changing now.
Messed up NOAA number: 12423 C5.1 2015-09-29 04:46:00 should be 12428 ; changing now.
Messed up NOAA number: 12423 M2.9 2015-09-29 05:16:00 should be 12422 ; changing now.
Messed up NOAA number: 12565 M5.0 2016-07-23 02:11:00 

In [15]:
# Case 3: Updating flare peak time.

# create an empty array to hold row numbers to drop at the end
flares_drops = []
active_region_numbers_noaa = [item["ar_noaanum"]
                              for item in listofresults["hek"]]
flare_classes_noaa = [item["fl_goescls"] for item in listofresults["hek"]]

for i in range(flares.shape[0]):
    # check if a particular DONKI flare peak time is also in the NOAA database
    peak_time_donki = flares['peakTime'].iloc[i]
    if not peak_time_donki in peak_times_noaa:
        active_region_number_donki = int(
            flares['activeRegionNum'].iloc[i])
        flare_class_donki = flares['classType'].iloc[i]
        flare_class_indices = [i for i, x in enumerate(
            flare_classes_noaa) if x == flare_class_donki]
        active_region_indices = [i for i, x in enumerate(
            active_region_numbers_noaa) if x == active_region_number_donki]
        common_indices = list(
            set(flare_class_indices).intersection(active_region_indices))
        if common_indices:
            print("Messed up time:", int(flares['activeRegionNum'].iloc[i]), flares['classType'].iloc[i],
                  flares['peakTime'].iloc[i], "should be", peak_times_noaa[common_indices[0]], "; changing now.")
            flares['peakTime'].iloc[i] = peak_times_noaa[common_indices[0]]
            number_of_donki_mistakes += 1
        if not common_indices:
            print("DONKI flare peak time",
                  flares['peakTime'].iloc[i], "has no match; dropping row.")
            flares_drops.append(i)
            number_of_donki_mistakes += 1

# Drop the rows for which the NOAA active region number and flare class associated with
# the messed-up flare peak time in the DONKI database has no match in the GOES flare database
flares = flares.drop(flares_drops)
only_flares = flares.reset_index(drop=True)

# Create a list of corrected flare peak times
peak_times_donki = [flares['peakTime'].iloc[i]
                    for i in range(flares.shape[0])]

Messed up time: 11967 M2.1 2014-01-14 06:39:00 should be 2014-01-30 06:39:00.000 ; changing now.
DONKI flare peak time 2014-01-28 15:26:00 has no match; dropping row.
DONKI flare peak time 2014-02-02 16:36:00 has no match; dropping row.
Messed up time: 11967 M3.1 2014-02-02 18:18:00 should be 2014-02-02 18:11:00.000 ; changing now.
Messed up time: 11967 M1.3 2014-02-02 22:14:00 should be 2014-02-02 22:04:00.000 ; changing now.
DONKI flare peak time 2014-02-04 03:06:00 has no match; dropping row.
Messed up time: 11967 M1.5 2014-02-06 23:03:00 should be 2014-01-28 04:09:00.000 ; changing now.
Messed up time: 11967 M2.0 2014-02-07 04:53:00 should be 2014-02-07 04:56:00.000 ; changing now.
Messed up time: 11974 M2.3 2014-02-14 02:53:00 should be 2014-02-14 02:57:00.000 ; changing now.
DONKI flare peak time 2014-06-10 07:00:00 has no match; dropping row.
Messed up time: 12166 M2.1 2014-09-11 15:25:00 should be 2014-09-11 15:26:00.000 ; changing now.
Messed up time: 12192 X1.1 2014-10-19 05:

In [ ]:
only_flares.to_csv('only_flares_base.csv')

#### flares with CME and SEP

In [11]:
# Case 1: filling nan ARs
number_of_donki_mistakes = 0  # count the number of DONKI mistakes
# create an empty array to hold row numbers to drop at the end
flare_linked_drops = []

for i in range(flare_linked.shape[0]):
    if (np.isnan(flare_linked.loc[i]['activeRegionNum'])):
        time = flare_linked['peakTime'].iloc[i]
        time_range = TimeRange(time, time)
        listofresult = Fido.search(a.Time(time_range),a.hek.EventType("FL"),a.hek.OBS.Observatory == "GOES")

        if len(listofresult["hek"]) == 0:
            print(flare_linked.loc[i]['classType'], "has no match in the GOES flare database ; dropping row.")
            flare_linked_drops.append(i)
            number_of_donki_mistakes += 1
            continue
        else:
            if (listofresult[0]['ar_noaanum'] == 0).all():
                print(flare_linked.loc[i]['activeRegionNum'], flare_linked.loc[i]
                    ['classType'], "has no match in the GOES flare database ; dropping row.")
                flare_linked_drops.append(i)
                number_of_donki_mistakes += 1
                continue
            else:
                print("Missing NOAA number:", flare_linked['activeRegionNum'].iloc[i], flare_linked['classType'].iloc[i],
                    flare_linked['peakTime'].iloc[i], "should be", listofresult[0]['ar_noaanum'][0], "; changing now.")
                flare_linked['activeRegionNum'].iloc[i] = listofresult[0]['ar_noaanum']
                number_of_donki_mistakes += 1

# Drop the rows for which there is no active region number in both the DONKI and GOES flare databases
flare_linked = flare_linked.drop(flare_linked_drops)
flare_linked = flare_linked.reset_index(drop=True)
print('There are', number_of_donki_mistakes, 'DONKI mistakes so far.')

Missing NOAA number: nan X1.4 2011-09-22 11:01:00 should be 11302 ; changing now.
Missing NOAA number: nan X1.3 2012-03-07 01:14:00 should be 11430 ; changing now.
Missing NOAA number: nan M6.3 2012-03-09 03:53:00 should be 11429 ; changing now.
Missing NOAA number: nan M5.1 2012-05-17 01:47:00 should be 11476 ; changing now.
Missing NOAA number: nan X1.1 2012-07-06 23:08:00 should be 11515 ; changing now.
Missing NOAA number: nan M6.2 2012-07-28 20:56:00 should be 11532 ; changing now.
Missing NOAA number: nan M1.7 2012-11-08 02:23:00 should be 11611 ; changing now.
Missing NOAA number: nan M1.2 2013-03-15 06:58:00 should be 11692 ; changing now.
Missing NOAA number: nan X1.6 2013-05-13 02:17:00 should be 11748 ; changing now.
Missing NOAA number: nan X2.8 2013-05-13 16:05:00 should be 11748 ; changing now.
Missing NOAA number: nan X3.2 2013-05-14 01:11:00 should be 11748 ; changing now.
Missing NOAA number: nan X1.2 2013-05-15 01:48:00 should be 11748 ; changing now.
Missing NOAA num

In [14]:
# Case 2: Updating Incorrect AR
peak_times_noaa = [item["event_peaktime"] for item in listofresults["hek"]]

for i in range(flare_linked.shape[0]):
    # check if a particular DONKI flare peak time is also in the NOAA database
    peak_time_donki = flare_linked['peakTime'].iloc[i]
    if peak_time_donki in peak_times_noaa:
        index = peak_times_noaa.index(peak_time_donki)
    else:
        continue
    # ignore NOAA active region numbers equal to zero
    if (listofresults["hek"][index]['ar_noaanum'] == 0):
        continue
    # if yes, check if the DONKI and NOAA active region numbers match up for this peak time
    # if they don't, flag this peak time and replace the DONKI number with the NOAA number
    if (listofresults["hek"][index]['ar_noaanum'] != int(flare_linked['activeRegionNum'].iloc[i])):
        print('Messed up NOAA number:', int(flare_linked['activeRegionNum'].iloc[i]), flare_linked['classType'].iloc[i],
              flare_linked['peakTime'].iloc[i], "should be", listofresults["hek"][index]['ar_noaanum'], "; changing now.")
        flare_linked['activeRegionNum'].iloc[i] = listofresults["hek"][index]['ar_noaanum']
        number_of_donki_mistakes += 1
print('There are', number_of_donki_mistakes, 'DONKI mistakes so far.')

Messed up NOAA number: 11093 C4.4 2010-08-14 10:05:00 should be 11099 ; changing now.
Messed up NOAA number: 11099 C4.5 2010-08-18 05:48:00 should be 1099 ; changing now.
Messed up NOAA number: 11560 C8.4 2012-08-31 20:43:00 should be 11562 ; changing now.
Messed up NOAA number: 11943 X1.2 2014-01-07 18:32:00 should be 11944 ; changing now.
Messed up NOAA number: 12051 M1.2 2014-05-07 16:29:00 should be 12055 ; changing now.
Messed up NOAA number: 12160 M1.4 2014-07-01 11:23:00 should be 12106 ; changing now.
Messed up NOAA number: 12282 M2.4 2015-02-09 23:35:00 should be 12280 ; changing now.
Messed up NOAA number: 12321 M1.1 2015-04-23 10:07:00 should be 12322 ; changing now.
Messed up NOAA number: 12565 M7.6 2016-07-23 05:16:00 should be 12567 ; changing now.
Messed up NOAA number: 12565 M5.5 2016-07-23 05:31:00 should be 12567 ; changing now.
Messed up NOAA number: 12822 B1.2 2021-05-19 03:07:00 should be 12824 ; changing now.
Messed up NOAA number: 12822 B1.3 2021-05-19 17:08:00 s

In [15]:
# Case 3: Updating flare peak time.

# create an empty array to hold row numbers to drop at the end
flare_linked_drops = []

active_region_numbers_noaa = [item["ar_noaanum"]
                              for item in listofresults["hek"]]
flare_classes_noaa = [item["fl_goescls"] for item in listofresults["hek"]]

for i in range(flare_linked.shape[0]):
    # check if a particular DONKI flare peak time is also in the NOAA database
    peak_time_donki = flare_linked['peakTime'].iloc[i]
    if not peak_time_donki in peak_times_noaa:
        active_region_number_donki = int(
            flare_linked['activeRegionNum'].iloc[i])
        flare_class_donki = flare_linked['classType'].iloc[i]
        flare_class_indices = [i for i, x in enumerate(
            flare_classes_noaa) if x == flare_class_donki]
        active_region_indices = [i for i, x in enumerate(
            active_region_numbers_noaa) if x == active_region_number_donki]
        common_indices = list(
            set(flare_class_indices).intersection(active_region_indices))
        if common_indices:
            print("Messed up time:", int(flare_linked['activeRegionNum'].iloc[i]), flare_linked['classType'].iloc[i],
                  flare_linked['peakTime'].iloc[i], "should be", peak_times_noaa[common_indices[0]], "; changing now.")
            flare_linked['peakTime'].iloc[i] = peak_times_noaa[common_indices[0]]
            number_of_donki_mistakes += 1
        if not common_indices:
            print("DONKI flare peak time",
                  flare_linked['peakTime'].iloc[i], "has no match; dropping row.")
            flare_linked_drops.append(i)
            number_of_donki_mistakes += 1

# Drop the rows for which the NOAA active region number and flare class associated with
# the messed-up flare peak time in the DONKI database has no match in the GOES flare database
flare_linked = flare_linked.drop(flare_linked_drops)
flare_linked = flare_linked.reset_index(drop=True)

Messed up time: 11429 X1.1 2012-03-05 04:05:00 should be 2012-03-05 04:09:00.000 ; changing now.
DONKI flare peak time 2012-03-10 17:27:00 has no match; dropping row.
Messed up time: 11745 M5.0 2013-05-22 13:38:00 should be 2013-05-22 13:32:00.000 ; changing now.
DONKI flare peak time 2014-02-09 16:14:00 has no match; dropping row.
Messed up time: 12127 M1.5 2014-08-01 18:12:00 should be 2014-08-01 18:13:00.000 ; changing now.
Messed up time: 12146 M2.0 2014-08-25 15:10:00 should be 2014-08-25 15:11:00.000 ; changing now.
DONKI flare peak time 2014-09-03 13:53:00 has no match; dropping row.
DONKI flare peak time 2014-09-09 00:28:00 has no match; dropping row.
Messed up time: 12172 M2.3 2014-09-23 23:15:00 should be 2014-09-23 23:16:00.000 ; changing now.
Messed up time: 12242 X1.8 2014-12-20 00:24:00 should be 2014-12-20 00:28:00.000 ; changing now.
DONKI flare peak time 2015-05-12 03:22:00 has no match; dropping row.
Messed up time: 12445 M1.9 2015-11-04 03:25:00 should be 2015-11-04 

In [ ]:
flare_linked.to_csv("flare_linked_base.csv")

#### only CMEs

In [17]:
only_cme = cme

In [ ]:
only_cme.to_csv("only_cme_base.csv")

## data

In [ ]:
only_flares = pd.read_csv("only_flares_base.csv")
flare_linked = pd.read_csv("flare_linked_base.csv")
only_cme = pd.read_csv("only_cme_base.csv")

In [19]:
for i in range(only_flares.shape[0]):
    only_flares['peakTime'].iloc[i] = convert_to_datetime(only_flares['peakTime'].iloc[i])
for i in range(only_cme.shape[0]):
    only_cme['startTime'].iloc[i] = convert_to_datetime(only_cme['startTime'].iloc[i])
for i in range(flare_linked.shape[0]):
    flare_linked['peakTime'].iloc[i] = convert_to_datetime(flare_linked['peakTime'].iloc[i])

### Fetching JSOC Data

In [16]:
jsoc = pd.read_csv('http://jsoc.stanford.edu/doc/data/hmi/harpnum_to_noaa/all_harps_with_noaa_ars.txt', sep=' ')

In [20]:
def get_the_jsoc_data(event_count, t_rec):
    """
    Parameters
    ----------
    event_count: number of events
                 int

    t_rec:       list of times, one associated with each event in event_count
                 list of strings in JSOC format ('%Y.%m.%d_%H:%M_TAI')

    """

    catalog_data = []
    classification = []

    for i in range(event_count):

        print("=====", i, "=====")
        # next match NOAA_ARS to HARPNUM
        idx = jsoc[jsoc['NOAA_ARS'].str.contains(
            str(int(listofactiveregions[i])))]

        # if there's no HARPNUM, quit
        if (idx.empty == True):
            print('skip: there are no matching HARPNUMs for',
                  str(int(listofactiveregions[i])))
            continue

        # construct jsoc_info queries and query jsoc database; we are querying for 25 keywords
        url = "http://jsoc.stanford.edu/cgi-bin/ajax/jsoc_info?ds=hmi.sharp_720s["+str(
            idx.HARPNUM.values[0])+"]["+t_rec[i]+"][? (CODEVER7 !~ '1.1 ') and (abs(OBS_VR)< 3500) and (QUALITY<65536) ?]&op=rs_list&key=USFLUX,MEANGBT,MEANJZH,MEANPOT,SHRGT45,TOTUSJH,MEANGBH,MEANALP,MEANGAM,MEANGBZ,MEANJZD,TOTUSJZ,SAVNCPP,TOTPOT,MEANSHR,AREA_ACR,R_VALUE,ABSNJZH"
        response = requests.get(url)

        # if there's no response at this time, quit
        if response.status_code != 200:
            print('skip: cannot successfully get an http response')
            continue

        # read the JSON output
        data = response.json()

        # if there are no data at this time, quit
        if data['count'] == 0:
            print('skip: there are no data for HARPNUM',
                  idx.HARPNUM.values[0], 'at time', t_rec[i])
            continue

        # check to see if the active region is too close to the limb
        # we can compute the latitude of an active region in stonyhurst coordinates as follows:
        # latitude_stonyhurst = CRVAL1 - CRLN_OBS
        # for this we have to query the CEA series (but above we queried the other series as the CEA series does not have CODEVER5 in it)

        url = "http://jsoc.stanford.edu/cgi-bin/ajax/jsoc_info?ds=hmi.sharp_cea_720s["+str(
            idx.HARPNUM.values[0])+"]["+t_rec[i]+"][? (abs(OBS_VR)< 3500) and (QUALITY<65536) ?]&op=rs_list&key=CRVAL1,CRLN_OBS"
        response = requests.get(url)

        # if there's no response at this time, quit
        if response.status_code != 200:
            print('skip: failed to find CEA JSOC data for HARPNUM',
                  idx.HARPNUM.values[0], 'at time', t_rec[i])
            continue

        # read the JSON output
        latitude_information = response.json()

        # if there are no data at this time, quit
        if latitude_information['count'] == 0:
            print('skip: there are no data for HARPNUM',
                  idx.HARPNUM.values[0], 'at time', t_rec[i])
            continue

        CRVAL1 = float(latitude_information['keywords'][0]['values'][0])
        CRLN_OBS = float(latitude_information['keywords'][1]['values'][0])
        if (np.absolute(CRVAL1 - CRLN_OBS) > 70.0):
            print('skip: latitude is out of range for HARPNUM',
                  idx.HARPNUM.values[0], 'at time', t_rec[i])
            continue

        if ('MISSING' in str(data['keywords'])):
            print('skip: there are some missing keywords for HARPNUM',
                  idx.HARPNUM.values[0], 'at time', t_rec[i])
            continue

        print('accept NOAA Active Region number', str(int(
            listofactiveregions[i])), 'and HARPNUM', idx.HARPNUM.values[0], 'at time', t_rec[i])

        individual_flare_data = []
        for j in range(18):
            individual_flare_data.append(
                float(data['keywords'][j]['values'][0]))

        catalog_data.append(list(individual_flare_data))

        single_class_instance = [idx.HARPNUM.values[0], str(
            int(listofactiveregions[i])), listofgoesclasses[i], t_rec[i]]
        classification.append(single_class_instance)

    return catalog_data, classification

In [ ]:
for j in [12,24,36,48,60,72,84,96,108,120]:
    timedelayvariable = j
    print("Timedelay variable:", timedelayvariable)
    t_rec = [(only_flares['peakTime'].iloc[i] - timedelta(hours=timedelayvariable)).strftime('%Y.%m.%d_%H:%M_TAI') for i in range(only_flares.shape[0])]

    listofactiveregions = list(only_flares['activeRegionNum'].values.flatten())
    listofgoesclasses = list(only_flares['classType'].values.flatten())

    result = get_the_jsoc_data(only_flares.shape[0], t_rec)
    df1 = pd.DataFrame(result[0], columns=['USFLUX', 'MEANGBT', 'MEANJZH', 'MEANPOT', 'SHRGT45', 'TOTUSJH',
                                          'MEANGBH', 'MEANALP', 'MEANGAM', 'MEANGBZ', 'MEANJZD', 'TOTUSJZ', 'SAVNCPP',
                                          'TOTPOT', 'MEANSHR', 'AREA_ACR', 'R_VALUE', 'ABSNJZH'])
    df2 = pd.DataFrame(result[1], columns=['HARPNUM', 'NOAA_ARS', 'GOES_CLASS', 'PEAK_TIME'])
    df = pd.concat([df1, df2], axis=1)
    df.to_csv("Data 2024/only_flares_" + str(timedelayvariable) + ".csv", index=False)

Streaming output truncated to the last 5000 lines.
accept NOAA Active Region number 13181 and HARPNUM 8942 at time 2023.01.08_22:15_TAI
===== 531 =====
accept NOAA Active Region number 13182 and HARPNUM 8950 at time 2023.01.10_08:21_TAI
===== 532 =====
skip: there are no data for HARPNUM 8977 at time 2023.01.11_02:31_TAI
===== 533 =====
skip: latitude is out of range for HARPNUM 8977 at time 2023.01.13_11:29_TAI
===== 534 =====
skip: latitude is out of range for HARPNUM 8977 at time 2023.01.13_22:35_TAI
===== 535 =====
skip: latitude is out of range for HARPNUM 8978 at time 2023.01.14_16:03_TAI
===== 536 =====
skip: latitude is out of range for HARPNUM 8978 at time 2023.01.14_22:12_TAI
===== 537 =====
accept NOAA Active Region number 13190 and HARPNUM 8977 at time 2023.01.18_00:26_TAI
===== 538 =====
accept NOAA Active Region number 13194 and HARPNUM 8971 at time 2023.01.18_05:02_TAI
===== 539 =====
accept NOAA Active Region number 13190 and HARPNUM 8977 at time 2023.01.20_22:11_TAI
==

In [ ]:
for j in [12,24,36,48,60,72,84,96,108,120]:
    timedelayvariable = j
    print("Timedelay variable:", timedelayvariable)
    t_rec = [(flare_linked['peakTime'].iloc[i] - timedelta(hours=timedelayvariable)).strftime('%Y.%m.%d_%H:%M_TAI') for i in range(flare_linked.shape[0])]

    listofactiveregions = list(flare_linked['activeRegionNum'].values.flatten())
    listofgoesclasses = list(flare_linked['classType'].values.flatten())

    result = get_the_jsoc_data(flare_linked.shape[0], t_rec)
    df1 = pd.DataFrame(result[0], columns=['USFLUX', 'MEANGBT', 'MEANJZH', 'MEANPOT', 'SHRGT45', 'TOTUSJH',
                                          'MEANGBH', 'MEANALP', 'MEANGAM', 'MEANGBZ', 'MEANJZD', 'TOTUSJZ', 'SAVNCPP',
                                          'TOTPOT', 'MEANSHR', 'AREA_ACR', 'R_VALUE', 'ABSNJZH'])
    df2 = pd.DataFrame(result[1], columns=['HARPNUM', 'NOAA_ARS', 'GOES_CLASS', 'PEAK_TIME'])
    df = pd.concat([df1, df2], axis=1)
    df.to_csv("Data 2024/flare_linked_" + str(timedelayvariable) + ".csv", index=False)

Streaming output truncated to the last 5000 lines.
===== 0 =====
accept NOAA Active Region number 11099 and HARPNUM 115 at time 2010.08.10_10:05_TAI
===== 1 =====
accept NOAA Active Region number 1099 and HARPNUM 115 at time 2010.08.14_05:48_TAI
===== 2 =====
accept NOAA Active Region number 11158 and HARPNUM 377 at time 2011.02.11_01:56_TAI
===== 3 =====
skip: there are no data for HARPNUM 392 at time 2011.02.20_07:35_TAI
===== 4 =====
skip: latitude is out of range for HARPNUM 401 at time 2011.03.03_14:30_TAI
===== 5 =====
accept NOAA Active Region number 11164 and HARPNUM 393 at time 2011.03.03_20:12_TAI
===== 6 =====
skip: there are no data for HARPNUM 415 at time 2011.03.04_03:58_TAI
===== 7 =====
skip: there are some missing keywords for HARPNUM 637 at time 2011.06.03_06:41_TAI
===== 8 =====
accept NOAA Active Region number 11261 and HARPNUM 750 at time 2011.07.30_13:48_TAI
===== 9 =====
accept NOAA Active Region number 11261 and HARPNUM 750 at time 2011.07.31_03:57_TAI
===== 10 

In [22]:
def get_the_jsoc_data(event_count, t_rec):
    """
    Parameters
    ----------
    event_count: number of events
                 int

    t_rec:       list of times, one associated with each event in event_count
                 list of strings in JSOC format ('%Y.%m.%d_%H:%M_TAI')

    """

    catalog_data = []
    classification = []

    for i in range(event_count):

        print("=====", i, "=====")
        # next match NOAA_ARS to HARPNUM
        idx = jsoc[jsoc['NOAA_ARS'].str.contains(
            str(int(listofactiveregions[i])))]

        # if there's no HARPNUM, quit
        if (idx.empty == True):
            print('skip: there are no matching HARPNUMs for',
                  str(int(listofactiveregions[i])))
            continue

        # construct jsoc_info queries and query jsoc database; we are querying for 25 keywords
        url = "http://jsoc.stanford.edu/cgi-bin/ajax/jsoc_info?ds=hmi.sharp_720s["+str(
            idx.HARPNUM.values[0])+"]["+t_rec[i]+"][? (CODEVER7 !~ '1.1 ') and (abs(OBS_VR)< 3500) and (QUALITY<65536) ?]&op=rs_list&key=USFLUX,MEANGBT,MEANJZH,MEANPOT,SHRGT45,TOTUSJH,MEANGBH,MEANALP,MEANGAM,MEANGBZ,MEANJZD,TOTUSJZ,SAVNCPP,TOTPOT,MEANSHR,AREA_ACR,R_VALUE,ABSNJZH"
        response = requests.get(url)

        # if there's no response at this time, quit
        if response.status_code != 200:
            print('skip: cannot successfully get an http response')
            continue

        # read the JSON output
        data = response.json()

        # if there are no data at this time, quit
        if data['count'] == 0:
            print('skip: there are no data for HARPNUM',
                  idx.HARPNUM.values[0], 'at time', t_rec[i])
            continue

        # check to see if the active region is too close to the limb
        # we can compute the latitude of an active region in stonyhurst coordinates as follows:
        # latitude_stonyhurst = CRVAL1 - CRLN_OBS
        # for this we have to query the CEA series (but above we queried the other series as the CEA series does not have CODEVER5 in it)

        url = "http://jsoc.stanford.edu/cgi-bin/ajax/jsoc_info?ds=hmi.sharp_cea_720s["+str(
            idx.HARPNUM.values[0])+"]["+t_rec[i]+"][? (abs(OBS_VR)< 3500) and (QUALITY<65536) ?]&op=rs_list&key=CRVAL1,CRLN_OBS"
        response = requests.get(url)

        # if there's no response at this time, quit
        if response.status_code != 200:
            print('skip: failed to find CEA JSOC data for HARPNUM',
                  idx.HARPNUM.values[0], 'at time', t_rec[i])
            continue

        # read the JSON output
        latitude_information = response.json()

        # if there are no data at this time, quit
        if latitude_information['count'] == 0:
            print('skip: there are no data for HARPNUM',
                  idx.HARPNUM.values[0], 'at time', t_rec[i])
            continue

        CRVAL1 = float(latitude_information['keywords'][0]['values'][0])
        CRLN_OBS = float(latitude_information['keywords'][1]['values'][0])
        if (np.absolute(CRVAL1 - CRLN_OBS) > 70.0):
            print('skip: latitude is out of range for HARPNUM',
                  idx.HARPNUM.values[0], 'at time', t_rec[i])
            continue

        if ('MISSING' in str(data['keywords'])):
            print('skip: there are some missing keywords for HARPNUM',
                  idx.HARPNUM.values[0], 'at time', t_rec[i])
            continue

        print('accept NOAA Active Region number', str(int(
            listofactiveregions[i])), 'and HARPNUM', idx.HARPNUM.values[0], 'at time', t_rec[i])

        individual_flare_data = []
        for j in range(18):
            individual_flare_data.append(
                float(data['keywords'][j]['values'][0]))

        catalog_data.append(list(individual_flare_data))

        single_class_instance = [idx.HARPNUM.values[0], str(
            int(listofactiveregions[i])), listofcmeanalysis[i], t_rec[i]]
        classification.append(single_class_instance)

    return catalog_data, classification

In [ ]:
for j in [12,24,36,48,60,72,84,96,108,120]:
    timedelayvariable = j
    print("Timedelay variable:", timedelayvariable)
    t_rec = [(only_cme['startTime'].iloc[i] - timedelta(hours=timedelayvariable)).strftime('%Y.%m.%d_%H:%M_TAI') for i in range(only_cme.shape[0])]

    listofactiveregions = list(only_cme['activeRegionNum'].values.flatten())
    listofcmeanalysis = list(only_cme['cmeAnalyses'].values.flatten())

    result = get_the_jsoc_data(only_cme.shape[0], t_rec)
    df1 = pd.DataFrame(result[0], columns=['USFLUX', 'MEANGBT', 'MEANJZH', 'MEANPOT', 'SHRGT45', 'TOTUSJH',
                                          'MEANGBH', 'MEANALP', 'MEANGAM', 'MEANGBZ', 'MEANJZD', 'TOTUSJZ', 'SAVNCPP',
                                          'TOTPOT', 'MEANSHR', 'AREA_ACR', 'R_VALUE', 'ABSNJZH'])
    df2 = pd.DataFrame(result[1], columns=['HARPNUM', 'NOAA_ARS', 'ANALYSIS', 'PEAK_TIME'])
    df = pd.concat([df1, df2], axis=1)
    df.to_csv("Data 2024/only_cme_" + str(timedelayvariable) + ".csv", index=False)

Streaming output truncated to the last 5000 lines.
===== 512 =====
accept NOAA Active Region number 13637 and HARPNUM 11079 at time 2024.04.17_21:24_TAI
===== 513 =====
accept NOAA Active Region number 13638 and HARPNUM 11079 at time 2024.04.18_00:48_TAI
===== 514 =====
accept NOAA Active Region number 13638 and HARPNUM 11079 at time 2024.04.20_07:24_TAI
===== 515 =====
accept NOAA Active Region number 13638 and HARPNUM 11079 at time 2024.04.20_08:48_TAI
===== 516 =====
accept NOAA Active Region number 13638 and HARPNUM 11079 at time 2024.04.20_10:12_TAI
===== 517 =====
skip: latitude is out of range for HARPNUM 11142 at time 2024.04.28_11:36_TAI
===== 518 =====
accept NOAA Active Region number 13663 and HARPNUM 11142 at time 2024.04.28_17:36_TAI
===== 519 =====
accept NOAA Active Region number 13663 and HARPNUM 11142 at time 2024.04.29_03:12_TAI
===== 520 =====
accept NOAA Active Region number 13663 and HARPNUM 11142 at time 2024.05.02_15:48_TAI
===== 521 =====
accept NOAA Active Regi

### Data

In [ ]:
flare_linked = pd.read_csv("jsoc/flare_linked_12.csv")
only_cme = pd.read_csv("jsoc/only_cme_12.csv")
only_flares = pd.read_csv("jsoc/only_flares_12.csv")

In [ ]:
flare_linked

,USFLUX,MEANGBT,MEANJZH,MEANPOT,SHRGT45,TOTUSJH,MEANGBH,MEANALP,MEANGAM,MEANGBZ,...,SAVNCPP,TOTPOT,MEANSHR,AREA_ACR,R_VALUE,ABSNJZH,HARPNUM,NOAA_ARS,GOES_CLASS,PEAK_TIME
0,1.442016e+22,104.249,-0.007450,5357.871,25.487,1010.143,52.438,-0.019349,41.182,105.795,...,6.601919e+12,1.589956e+23,34.327,488.760559,3.807,166.467,115,11099,C4.4,2010.08.13_22:05_TAI
1,2.416427e+22,93.429,0.013943,15371.740,59.408,2940.413,68.471,0.030222,60.629,101.466,...,9.554389e+12,8.171862e+23,52.303,1078.161865,4.829,558.116,377,11158,X2.2,2011.02.14_13:56_TAI
2,2.316485e+22,96.645,-0.005333,11578.920,48.318,1829.461,61.270,-0.010615,53.676,101.450,...,9.006182e+12,5.367295e+23,45.522,946.170837,4.352,186.149,401,11166,M2.0,2011.03.07_02:30_TAI
3,6.534931e+22,85.268,0.002820,8713.998,24.840,4221.041,45.785,0.006022,39.556,94.534,...,1.857044e+13,1.046693e+24,33.991,1927.615967,4.976,255.077,393,11164,M3.7,2011.03.07_08:12_TAI
4,2.467217e+22,106.978,0.008833,3060.831,9.296,1419.924,43.043,0.028216,31.113,107.913,...,1.961166e+13,1.577271e+23,25.680,835.504700,3.859,342.725,637,11226,M2.5,2011.06.06_18:41_TAI
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
104,5.604837e+22,92.444,-0.032351,15655.120,37.497,5831.543,57.427,-0.052533,45.804,106.127,...,6.328512e+13,1.494623e+24,40.415,1898.472900,5.069,2325.598,7115,12673,M3.9,2017.09.07_11:59_TAI
105,5.613295e+22,93.027,-0.038322,15304.870,35.437,5842.066,57.352,-0.061129,45.110,106.716,...,7.511389e+13,1.433866e+24,39.519,1905.817749,5.048,2703.341,7115,12673,M1.2,2017.09.07_15:43_TAI
106,5.806990e+22,89.143,-0.031928,13799.640,34.294,5644.398,54.580,-0.053679,44.496,102.628,...,6.727857e+13,1.362156e+24,38.499,1908.245361,4.998,2373.013,7115,12673,M8.1,2017.09.07_19:49_TAI
107,6.273545e+21,75.352,0.003173,11195.290,42.382,316.479,41.020,0.006211,49.238,77.549,...,3.159556e+12,1.431625e+23,41.883,184.446060,3.729,30.550,7131,12680,C3.0,2017.09.11_19:29_TAI


In [ ]:
only_cme

,USFLUX,MEANGBT,MEANJZH,MEANPOT,SHRGT45,TOTUSJH,MEANGBH,MEANALP,MEANGAM,MEANGBZ,...,SAVNCPP,TOTPOT,MEANSHR,AREA_ACR,R_VALUE,ABSNJZH,HARPNUM,NOAA_ARS,ANALYSIS,PEAK_TIME
0,1.507124e+22,135.582,0.007611,3604.044,13.767,1416.605,64.129,0.024348,36.539,137.686,...,1.369774e+13,1.159824e+23,28.462,1171.015503,4.183,184.440,245,11123,"[{'isMostAccurate': True, 'time21_5': '2010-11...",2010.11.11_05:00_TAI
1,3.702705e+22,96.900,-0.016211,15245.260,48.519,3225.808,62.505,-0.027524,53.618,104.427,...,3.478225e+13,1.070507e+24,46.091,1687.398438,4.741,857.096,892,11302,"[{'isMostAccurate': True, 'time21_5': '2011-10...",2011.09.30_21:54_TAI
2,9.887384e+21,101.142,-0.029578,15793.910,55.681,1023.174,69.053,-0.057004,61.357,102.247,...,1.826780e+13,3.583161e+23,50.751,583.930054,4.287,505.244,902,11305,"[{'isMostAccurate': True, 'time21_5': '2011-10...",2011.09.30_22:24_TAI
3,6.834983e+22,85.107,0.011764,11441.550,32.970,5162.636,49.803,0.022286,43.596,95.353,...,3.703731e+13,1.386199e+24,38.209,2301.692383,5.021,1073.172,3341,11890,"[{'isMostAccurate': True, 'time21_5': '2013-11...",2013.11.04_20:36_TAI
4,5.911507e+22,98.799,0.010249,12648.870,37.735,5431.100,58.838,0.019593,46.629,107.898,...,1.286541e+13,1.363498e+24,40.829,2448.140869,5.009,831.904,3341,11890,"[{'isMostAccurate': True, 'time21_5': '2013-11...",2013.11.06_02:36_TAI
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
96,1.992408e+22,74.509,-0.003792,11549.770,38.098,1015.662,36.767,-0.008381,46.388,78.145,...,2.822306e+11,4.832121e+23,40.502,631.375427,3.271,119.461,7148,12683,"[{'isMostAccurate': True, 'time21_5': '2017-09...",2017.09.28_14:39_TAI
97,2.079321e+22,76.411,-0.004909,8543.737,24.247,1194.607,36.824,-0.010898,38.743,83.660,...,6.969821e+12,3.391231e+23,31.733,515.013000,4.215,146.706,7148,12683,"[{'isMostAccurate': True, 'time21_5': '2017-10...",2017.10.06_20:48_TAI
98,1.507876e+22,98.186,0.001690,3996.114,12.717,885.901,41.912,0.004405,33.049,101.120,...,3.977853e+12,1.161387e+23,27.673,389.947601,3.981,36.980,7189,12687,"[{'isMostAccurate': True, 'time21_5': '2017-11...",2017.11.16_02:54_TAI
99,6.013340e+21,117.000,0.005289,3809.664,12.328,434.736,54.299,0.011717,34.750,120.592,...,2.236462e+12,4.157309e+22,28.237,165.846252,3.676,43.456,7211,12692,"[{'isMostAccurate': True, 'time21_5': '2017-12...",2017.12.20_12:12_TAI


In [ ]:
only_flares

,USFLUX,MEANGBT,MEANJZH,MEANPOT,SHRGT45,TOTUSJH,MEANGBH,MEANALP,MEANGAM,MEANGBZ,...,SAVNCPP,TOTPOT,MEANSHR,AREA_ACR,R_VALUE,ABSNJZH,HARPNUM,NOAA_ARS,GOES_CLASS,PEAK_TIME
0,2.492258e+21,136.018,-0.010670,3867.412,26.318,266.345,73.914,-0.028015,45.745,143.165,...,2.072907e+12,2.074468e+22,35.722,91.149933,3.231,43.086,54,11081,M2.0,2010.06.11_12:57_TAI
1,1.291028e+22,96.459,-0.005842,8250.659,24.915,869.410,42.809,-0.014357,39.289,96.397,...,1.866188e+12,2.194078e+23,34.030,510.853424,3.739,116.988,115,11093,M1.0,2010.08.07_06:24_TAI
2,1.186780e+22,131.391,0.002885,3811.955,16.530,993.808,63.644,0.008215,37.853,132.928,...,6.300825e+12,9.150875e+22,29.788,696.704041,4.000,52.141,211,11112,M2.9,2010.10.16_07:12_TAI
3,1.250015e+22,112.350,0.025216,9586.912,50.103,1814.469,83.279,0.067965,59.725,118.404,...,2.250187e+13,2.976450e+23,48.945,574.865662,4.656,589.496,377,11158,M6.6,2011.02.13_05:38_TAI
4,2.313220e+22,98.992,0.021898,15359.870,57.868,2858.902,73.307,0.045729,60.477,108.353,...,1.715663e+13,7.623972e+23,51.407,1008.172302,4.967,818.450,377,11158,M2.2,2011.02.14_05:26_TAI
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
232,5.930980e+22,80.144,-0.014250,12896.500,31.664,4921.651,49.486,-0.025893,43.627,94.480,...,2.556215e+13,1.332456e+24,36.976,1862.779663,4.885,1108.560,7115,12673,M2.1,2017.09.08_11:45_TAI
233,5.855636e+22,79.850,-0.015104,12481.560,31.432,4598.592,48.064,-0.025761,42.954,94.869,...,2.320097e+13,1.212606e+24,36.622,1854.639160,4.811,1104.857,7115,12673,M1.1,2017.09.08_16:28_TAI
234,5.855636e+22,79.850,-0.015104,12481.560,31.432,4598.592,48.064,-0.025761,42.954,94.869,...,2.320097e+13,1.212606e+24,36.622,1854.639160,4.811,1104.857,7115,12673,M1.1,2017.09.08_16:28_TAI
235,7.043642e+21,92.091,0.002661,9896.233,37.659,405.066,47.296,0.005628,46.528,93.091,...,2.944581e+12,1.401531e+23,39.585,245.657135,3.902,28.374,7131,12680,B1.4,2017.09.12_19:48_TAI
